# T1134.004 - Parent PID Spoofing
Adversaries may spoof the parent process identifier (PPID) of a new process to evade process-monitoring defenses or to elevate privileges. New processes are typically spawned directly from their parent, or calling, process unless explicitly specified. One way of explicitly assigning the PPID of a new process is via the <code>CreateProcess</code> API call, which supports a parameter that defines the PPID to use.(Citation: DidierStevens SelectMyParent Nov 2009) This functionality is used by Windows features such as User Account Control (UAC) to correctly set the PPID after a requested elevated process is spawned by SYSTEM (typically via <code>svchost.exe</code> or <code>consent.exe</code>) rather than the current user context.(Citation: Microsoft UAC Nov 2018)

Adversaries may abuse these mechanisms to evade defenses, such as those blocking processes spawning directly from Office documents, and analysis targeting unusual/potentially malicious parent-child process relationships, such as spoofing the PPID of [PowerShell](https://attack.mitre.org/techniques/T1059/001)/[Rundll32](https://attack.mitre.org/techniques/T1218/011) to be <code>explorer.exe</code> rather than an Office document delivered as part of [Spearphishing Attachment](https://attack.mitre.org/techniques/T1566/001).(Citation: CounterCept PPID Spoofing Dec 2018) This spoofing could be executed via [Visual Basic](https://attack.mitre.org/techniques/T1059/005) within a malicious Office document or any code that can perform [Native API](https://attack.mitre.org/techniques/T1106).(Citation: CTD PPID Spoofing Macro Mar 2019)(Citation: CounterCept PPID Spoofing Dec 2018)

Explicitly assigning the PPID may also enable elevated privileges given appropriate access rights to the parent process. For example, an adversary in a privileged user context (i.e. administrator) may spawn a new process and assign the parent as a process running as SYSTEM (such as <code>lsass.exe</code>), causing the new process to be elevated via the inherited access token.(Citation: XPNSec PPID Nov 2017)

## Atomic Tests

### Atomic Test #1 - Parent PID Spoofing using PowerShell
This test uses PowerShell to replicates how Cobalt Strike does ppid spoofing and masquerade a spawned process.
Upon execution, "Process C:\Program Files\Internet Explorer\iexplore.exe is spawned with pid ####" will be displayed and
calc.exe will be launched.

Credit to In Ming Loh (https://github.com/countercept/ppid-spoofing/blob/master/PPID-Spoof.ps1)

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: DLL to inject must exist on disk at specified location (#{dll_path})

##### Check Prereq Commands:
```powershell
if (Test-Path PathToAtomicsFolder\T1134.004\bin\calc.dll) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory (split-path PathToAtomicsFolder\T1134.004\bin\calc.dll) -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1134.004/bin/calc.dll" -OutFile "PathToAtomicsFolder\T1134.004\bin\calc.dll"

```
##### Description: PPID.ps1 must exist on disk at $PathToAtomicsFolder\T1134.004\src\PPID-Spoof.ps1

##### Check Prereq Commands:
```powershell
if (Test-Path $PathToAtomicsFolder\T1134.004\src\PPID-Spoof.ps1) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory (split-path $PathToAtomicsFolder\T1134.004\src\PPID-Spoof.ps1) -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1134.004/src/PPID-Spoof.ps1" -OutFile $PathToAtomicsFolder\T1134.004\src\PPID-Spoof.ps1

```

In [ ]:
Invoke-AtomicTest T1134.004 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
. $PathToAtomicsFolder\T1134.004\src\PPID-Spoof.ps1
$ppid=Get-Process explorer | select -expand id
PPID-Spoof -ppid $ppid -spawnto "C:\Program Files\Internet Explorer\iexplore.exe" -dllpath "PathToAtomicsFolder\T1134.004\bin\calc.dll"
```

In [ ]:
Invoke-AtomicTest T1134.004 -TestNumbers 1

#### Cleanup: 
```powershell
Stop-Process -Name "calculator" -ErrorAction Ignore
Stop-Process -Name "iexplore" -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1134.004 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Parent PID Spoofing - Spawn from Current Process
Spawns a powershell.exe process as a child of the current process.
**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: The AtomicTestHarnesses module must be installed and Start-ATHProcessUnderSpecificParent must be exported in the module.
##### Check Prereq Commands:
```powershell
$RequiredModule = Get-Module -Name AtomicTestHarnesses -ListAvailable
if (-not $RequiredModule) {exit 1}
if (-not $RequiredModule.ExportedCommands['Start-ATHProcessUnderSpecificParent']) {exit 1} else {exit 0}
```
##### Get Prereq Commands:
```powershell
Install-Module -Name AtomicTestHarnesses -Scope CurrentUser -Force

```

In [ ]:
Invoke-AtomicTest T1134.004 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Start-ATHProcessUnderSpecificParent -FilePath $Env:windir\System32\WindowsPowerShell\v1.0\powershell.exe -CommandLine '-Command Start-Sleep 10' -ParentId $PID```

In [ ]:
Invoke-AtomicTest T1134.004 -TestNumbers 2

### Atomic Test #3 - Parent PID Spoofing - Spawn from Specified Process
Spawns a notepad.exe process as a child of the current process.
**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: The AtomicTestHarnesses module must be installed and Start-ATHProcessUnderSpecificParent must be exported in the module.
##### Check Prereq Commands:
```powershell
$RequiredModule = Get-Module -Name AtomicTestHarnesses -ListAvailable
if (-not $RequiredModule) {exit 1}
if (-not $RequiredModule.ExportedCommands['Start-ATHProcessUnderSpecificParent']) {exit 1} else {exit 0}
```
##### Get Prereq Commands:
```powershell
Install-Module -Name AtomicTestHarnesses -Scope CurrentUser -Force

```

In [ ]:
Invoke-AtomicTest T1134.004 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Start-ATHProcessUnderSpecificParent  -ParentId $PID -TestGuid 12345678-1234-1234-1234-123456789123```

In [ ]:
Invoke-AtomicTest T1134.004 -TestNumbers 3

### Atomic Test #4 - Parent PID Spoofing - Spawn from svchost.exe
Spawnd a process as a child of the first accessible svchost.exe process.
**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: The AtomicTestHarnesses module must be installed and Start-ATHProcessUnderSpecificParent must be exported in the module.
##### Check Prereq Commands:
```powershell
$RequiredModule = Get-Module -Name AtomicTestHarnesses -ListAvailable
if (-not $RequiredModule) {exit 1}
if (-not $RequiredModule.ExportedCommands['Start-ATHProcessUnderSpecificParent']) {exit 1} else {exit 0}
```
##### Get Prereq Commands:
```powershell
Install-Module -Name AtomicTestHarnesses -Scope CurrentUser -Force

```

In [ ]:
Invoke-AtomicTest T1134.004 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Get-CimInstance -ClassName Win32_Process -Property Name, CommandLine, ProcessId -Filter "Name = 'svchost.exe' AND CommandLine LIKE '%'" | Select-Object -First 1 | Start-ATHProcessUnderSpecificParent -FilePath $Env:windir\System32\WindowsPowerShell\v1.0\powershell.exe -CommandLine '-Command Start-Sleep 10'```

In [ ]:
Invoke-AtomicTest T1134.004 -TestNumbers 4

### Atomic Test #5 - Parent PID Spoofing - Spawn from New Process
Creates a notepad.exe process and then spawns a powershell.exe process as a child of it.
**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: The AtomicTestHarnesses module must be installed and Start-ATHProcessUnderSpecificParent must be exported in the module.
##### Check Prereq Commands:
```powershell
$RequiredModule = Get-Module -Name AtomicTestHarnesses -ListAvailable
if (-not $RequiredModule) {exit 1}
if (-not $RequiredModule.ExportedCommands['Start-ATHProcessUnderSpecificParent']) {exit 1} else {exit 0}
```
##### Get Prereq Commands:
```powershell
Install-Module -Name AtomicTestHarnesses -Scope CurrentUser -Force

```

In [ ]:
Invoke-AtomicTest T1134.004 -TestNumbers 5 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Start-Process -FilePath $Env:windir\System32\notepad.exe -PassThru | Start-ATHProcessUnderSpecificParent -FilePath $Env:windir\System32\WindowsPowerShell\v1.0\powershell.exe -CommandLine '-Command Start-Sleep 10'```

In [ ]:
Invoke-AtomicTest T1134.004 -TestNumbers 5

## Detection
Look for inconsistencies between the various fields that store PPID information, such as the EventHeader ProcessId from data collected via Event Tracing for Windows (ETW), Creator Process ID/Name from Windows event logs, and the ProcessID and ParentProcessID (which are also produced from ETW and other utilities such as Task Manager and Process Explorer). The ETW provided EventHeader ProcessId identifies the actual parent process.(Citation: CounterCept PPID Spoofing Dec 2018)

Monitor and analyze API calls to <code>CreateProcess</code>/<code>CreateProcessA</code>, specifically those from user/potentially malicious processes and with parameters explicitly assigning PPIDs (ex: the Process Creation Flags of 0x8XXX, indicating that the process is being created with extended startup information(Citation: Microsoft Process Creation Flags May 2018)). Malicious use of <code>CreateProcess</code>/<code>CreateProcessA</code> may also be proceeded by a call to <code>UpdateProcThreadAttribute</code>, which may be necessary to update process creation attributes.(Citation: Secuirtyinbits Ataware3 May 2019) This may generate false positives from normal UAC elevation behavior, so compare to a system baseline/understanding of normal system activity if possible.